In [ ]:
from pathlib import Path

lines = Path("day11_input.txt").read_text().strip().splitlines()

outputs = {}
for line in lines:
    device, others = line.split(":")
    outputs[device] = others.strip().split(" ")

# Part 1

Find all paths from `you` to `out` in the device graph. Each device can be used at most
once per path, otherwise we would be stuck in infinite loops. We can use Depth-First
Search (DFS) to explore all possible paths.


In [ ]:
def dfs(current: str, target: str, path: set[str]) -> int:
    """Find number of paths from current to target avoiding cycles."""
    if current == target:
        return 1
    num_paths = 0
    for neighbor in outputs.get(current, []):
        if neighbor not in path:  # Avoid cycles
            num_paths += dfs(neighbor, target, path | {neighbor})
    return num_paths


dfs("you", "out", {"you"})

# Part 2

1. There is a lot of repetition in the paths, just look at the example. We can use
   memoization to cache the number of paths from each device to the goal. This way, when
   we reach a device we've already computed the paths for, we can simply add that number
   to our total instead of recomputing it.

2. There are no cycles in the graph. This means we don't need to keep track of the path
   of visited nodes.

3. There cannot be any paths where `dac` appears before `fft`. Because if there were
   both paths with `fft` before `dac` and paths with `dac` before `fft`, there would be
   cycles in the graph. Thus, we can split the problem into three parts:

   - Find the number of paths from `you` to `fft`
   - Find the number of paths from `fft` to `dac`
   - Find the number of paths from `dac` to `out`

   The total number of paths from `you` to `out` (that goes through `fft` and `dac`) is
   the product of those three numbers.


In [ ]:
from functools import cache


@cache
def dfs2(current: str, target: str) -> int:
    """Find number of paths from current to target."""
    if current == target:
        return 1
    num_paths = 0
    for neighbor in outputs.get(current, []):
        num_paths += dfs2(neighbor, target)
    return num_paths


dfs2("svr", "fft") * dfs2("fft", "dac") * dfs2("dac", "out")